In [ ]:
%load_ext autoreload
%autoreload 2

import pylupnt as pnt
import numpy as np
import matplotlib.pyplot as plt

output_dir = pnt.BASEDIR / "output" / "2025_FeatureMatching"

In [ ]:
config_ds = {
    "inherit_from": "datasets/unreal.yaml",
    "basedir": "/home/shared_ws6/data/unreal_engine/local_traverse/base_0",
    "cameras": ["front_left"],
}
dataset = pnt.datasets.Dataset.from_config(config_ds)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# 1. Data Extraction and Initialization
data = dataset[10]["cameras"]["front_left"]
rgb_img, depth_img, label_img = data["rgb"], data["depth"], data["label"]

# 2. Label Processing (Conversion and Colormap)
label_img = np.vectorize(pnt.UnrealDataset.LABEL_UNREAL_TO_LUPNT.get)(label_img)
unique_labels = np.unique(label_img)
label_to_idx = {j: i for i, j in enumerate(unique_labels)}
label_idx_img = np.vectorize(label_to_idx.get)(label_img)
colors = [plt.get_cmap("tab10")(i) for i in range(len(unique_labels))]
discrete_cmap = mcolors.ListedColormap(colors)

# 3. Depth Normalization (Corrected log_depth typo)
log_depth = np.log10(np.clip(depth_img, 1e-6, None))
log_min, log_max = np.floor(np.nanmin(log_depth[log_depth > 0])), np.ceil(
    np.nanmax(log_depth[log_depth > 0])
)
depth_ticks = np.arange(log_min, log_max + 1)

# 4. Figure and Axes Setup (Modification here: Removing fixed height from figsize)
fig_width = 12
fig, axes = plt.subplots(
    1,
    4,
    figsize=(fig_width, fig_width / 4),  # Estimate height for initialization
    gridspec_kw={"width_ratios": [1, 1, 1, 1]},
)

# --- Plot 1: RGB Image ---
plt.sca(axes[0])
plt.imshow(rgb_img)
plt.title("RGB")
plt.axis("off")

# --- Plot 2: Depth Image with Custom Colorbar ---
plt.sca(axes[1])
im1 = plt.imshow(log_depth, cmap="viridis", vmin=log_min, vmax=log_max)
plt.title("Depth")
plt.axis("off")
cax = inset_axes(
    axes[1],
    width="4%",
    height="40%",
    loc="lower left",
    bbox_to_anchor=(0.7, 0.09, 1, 1),
    bbox_transform=axes[1].transAxes,
)
cbar1 = fig.colorbar(im1, cax=cax, ticks=depth_ticks)
cbar1.ax.tick_params(labelsize="small", length=0, colors="white")
cbar1.set_ticklabels([f"$10^{{{int(tick)}}}$" for tick in depth_ticks])
cbar1.set_label("Depth [m]", fontsize="small", labelpad=8, color="white")
[label.set_color("white") for label in cbar1.ax.get_yticklabels()]

# --- Plot 3: Label Image with Legend ---
plt.sca(axes[2])
im2 = plt.imshow(label_idx_img, cmap=discrete_cmap, vmin=0, vmax=len(unique_labels) - 1)
plt.title("Label")
plt.axis("off")
label_names = [pnt.LABEL_NAMES[label] for label in unique_labels]
patches = [
    mpatches.Patch(color=colors[i], label=name, linewidth=1, alpha=1)
    for i, name in enumerate(label_names)
]
plt.legend(
    handles=patches,
    loc="upper right",
    bbox_to_anchor=(0.01, 0.01, 0.99, 0.99),
    bbox_transform=axes[2].transAxes,
    fontsize="small",
    ncol=2,
    frameon=True,
    borderaxespad=0.1,
)

# --- Plot 4: Trajectory (Modified for Limit Ratio) ---
plt.sca(axes[3])
plt.plot(dataset.positions[:, 1], dataset.positions[:, 0])
plt.scatter(dataset.positions[0, 1], dataset.positions[0, 0])
plt.xlabel("X [m]")
plt.ylabel("Y [m]")
plt.title("Trajectory")
plt.grid(True)
plt.axis("equal")

ratio = rgb_img.shape[0] / rgb_img.shape[1]
axes[3].set_box_aspect(ratio)
xlims, ylims = plt.xlim(), plt.ylim()
max_range = np.maximum(xlims[1] - xlims[0], (ylims[1] - ylims[0]) / ratio)
new_range_x, new_range_y = max_range, max_range * ratio
plt.xlim(np.mean(xlims) - new_range_x / 2, np.mean(xlims) + new_range_x / 2)
plt.ylim(np.mean(ylims) - new_range_y / 2, np.mean(ylims) + new_range_y / 2)

# --- Final Save and Display ---
plt.tight_layout(rect=[0, 0, 1, 1])
plt.savefig(output_dir / "figures/unreal_rgb_depth_label.pdf", dpi=300)
plt.show()

In [ ]:
def filter_features_by_depth(feats: pnt.Features, data: pnt.ImageData):
    u = feats.uv.astype(np.int32)
    depth = data.depth[u[:, 1], u[:, 0]]
    indices = np.where((depth > 0) & (depth < 1e3) & ~np.isnan(depth))[0]
    feats.filter(indices)
    return feats

In [ ]:
def plot_features(feats1, feats2, img_data1, img_data2, matches, filename=None):
    h, w = img_data1.rgb.shape[:2]

    # Check valid feature coordinates in image 1
    u1 = feats1.uv[:, 0].astype(np.int32).clip(0, w - 1)
    v1 = feats1.uv[:, 1].astype(np.int32).clip(0, h - 1)

    depth1 = img_data1.depth[v1, u1]

    # Convert to 3D world coordinates
    xyz_c1 = pnt.uv_to_xyz(feats1.uv, depth1, img_data1.intrinsics)
    xyz1_w = pnt.apply_transform(img_data1.world_T_cam, xyz_c1)

    # Project to image 2
    uv2_true, depth2 = pnt.xyz_to_uv(
        xyz1_w, img_data2.intrinsics, img_data2.world_T_cam, return_depth=True
    )

    # Filter valid projections
    valid_mask2: np.ndarray = (
        (depth1 > 0)
        & (depth2 > 0)
        & (0 <= uv2_true[:, 0])
        & (uv2_true[:, 0] < w)
        & (0 <= uv2_true[:, 1])
        & (uv2_true[:, 1] < h)
    )

    # Match distance
    gt_thresh = 0.01 * w
    dist_match = np.linalg.norm(
        uv2_true[matches.indexes[:, 0]] - feats2.uv[matches.indexes[:, 1]], axis=1
    )
    dists_normed = np.clip(dist_match / gt_thresh / 2.0, 0, 1)
    dists_normed = np.where(np.isnan(dists_normed), 1.0, dists_normed)
    match_colors = np.vstack(
        [dists_normed, 1 - dists_normed, np.zeros(len(dists_normed))]
    ).T

    # Precision and recall
    matched = np.zeros(len(feats1), dtype=np.bool)
    matched[matches.indexes[:, 0]] = True
    dist_feats1 = np.full(len(feats1), np.nan)
    dist_feats1[matches.indexes[:, 0]] = dist_match
    pos = (dist_feats1 < gt_thresh) & valid_mask2
    neg = (dist_feats1 >= gt_thresh) | ~valid_mask2
    tp = matched & pos
    fp = matched & neg
    fn = ~matched & pos
    prec = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    rec = np.sum(tp) / (np.sum(tp) + np.sum(fn))

    print(f"Positive: {np.sum(pos)}, Negative: {np.sum(neg)}")
    print(f"TP: {np.sum(tp)}, FP: {np.sum(fp)}, FN: {np.sum(fn)}")
    print(f"Precision: {prec:.2f}, Recall: {rec:.2f}")

    # Subsample matches
    n_matches = len(matches.indexes)
    sort_keys = np.lexsort(
        (matches.indexes[:, 1], matches.indexes[:, 0], matches.distances)
    )
    idxs_match = sort_keys[:n_matches]
    idxs_uv1, idxs_uv2 = matches.indexes[idxs_match].T

    pnp_config = {"threshold": 1.0, "confidence": 0.999, "max_iterations": 1000}
    pnp_solver = pnt.PnpSolver(pnp_config)
    K = pnt.make_camera_matrix(img_data2.intrinsics)
    pnp_result = pnp_solver.solve(
        xyz_c1[matches.indexes[:, 0]], feats2.uv[matches.indexes[:, 1]], K
    )

    # Compute error
    c2Tc1_pnp = pnp_result.tgt_T_src
    wTc1 = img_data1.world_T_cam
    c2Tw = pnt.invert_transform(img_data2.world_T_cam)
    c2Tc1_gt = c2Tw @ wTc1

    ea_t_pnp = np.linalg.norm(c2Tc1_gt[:3, 3] - c2Tc1_pnp[:3, 3])
    ea_R_pnp = pnt.rotation_angle(c2Tc1_pnp[:3, :3] @ c2Tc1_gt[:3, :3].T) * pnt.DEG

    print(f"Translation error: {ea_t_pnp:.2f} m")
    print(f"Rotation error: {ea_R_pnp:.2f} deg")

    h, w = img_data1["rgb"].shape[:2]
    ws = 10
    sep = np.ones((h, ws, 3))

    # Features
    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1.rgb, sep, img_data2.rgb]))
    plt.scatter(feats1.uv[:, 0], feats1.uv[:, 1], color="cyan", s=2)
    plt.scatter(feats2.uv[:, 0] + w + ws, feats2.uv[:, 1], color="magenta", s=2)
    plt.scatter([], [], color="cyan", s=10, label="Extracted Features 1")
    plt.scatter([], [], color="magenta", s=10, label="Extracted Features 2")
    plt.axis("off")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right", framealpha=1.0)
    plt.tight_layout()
    if filename is not None:
        plt.savefig(
            output_dir / f"figures/{filename}_all.pdf", dpi=300, bbox_inches="tight"
        )
    plt.show()

    # FP, FN, TP
    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1.rgb, sep, img_data2.rgb]))
    plt.scatter(feats1.uv[fp, 0], feats1.uv[fp, 1], color="red", s=2)
    plt.scatter(feats1.uv[fn, 0], feats1.uv[fn, 1], color="yellow", s=2)
    plt.scatter(feats1.uv[tp, 0], feats1.uv[tp, 1], color="lime", s=2)
    plt.scatter(
        uv2_true[tp & valid_mask2, 0] + w + ws,
        uv2_true[tp & valid_mask2, 1],
        color="lime",
        s=2,
    )
    plt.scatter(
        uv2_true[fp & valid_mask2, 0] + w + ws,
        uv2_true[fp & valid_mask2, 1],
        color="red",
        s=2,
    )
    plt.scatter(
        uv2_true[fn & valid_mask2, 0] + w + ws,
        uv2_true[fn & valid_mask2, 1],
        color="yellow",
        s=2,
    )
    plt.scatter([], [], color="lime", s=10, label=f"Correct ({np.sum(tp)} TP)")
    plt.scatter([], [], color="red", s=10, label=f"Incorrect ({np.sum(fp)} FP)")
    plt.scatter([], [], color="yellow", s=10, label=f"Missed ({np.sum(fn)} FN)")
    plt.scatter([], [], s=0.001, label=f"Precision: {prec:.2f}, Recall: {rec:.2f}")
    plt.axis("off")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right", framealpha=1.0)
    plt.tight_layout()
    if filename is not None:
        plt.savefig(
            output_dir / f"figures/{filename}_fp_fn.pdf", dpi=300, bbox_inches="tight"
        )
    plt.show()

    # Matches
    plt.figure(figsize=(10, 4))
    plt.imshow(np.hstack([img_data1.rgb, sep, img_data2.rgb]))
    # plt.scatter(feats1.uv[idxs_uv1, 0], feats1.uv[idxs_uv1, 1], color="cyan", s=10)
    # plt.scatter(feats2.uv[idxs_uv2, 0] + w + ws, feats2.uv[idxs_uv2, 1], color="cyan", s=10)
    for i, j, k in zip(idxs_uv1, idxs_uv2, idxs_match):
        pt1, pt2 = feats1.uv[i], feats2.uv[j]
        plt.plot(
            [pt1[0], pt2[0] + w + ws],
            [pt1[1], pt2[1]],
            "-",
            color=match_colors[k],
            lw=1.0,
            zorder=15,
        )
    plt.axis("off")
    plt.plot([], [], color="lime", lw=2.0, label="Low Error")
    plt.plot([], [], color="red", lw=2.0, label="High Error")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.legend(loc="upper right", framealpha=1.0)
    plt.tight_layout()
    if filename is not None:
        plt.savefig(
            output_dir / f"figures/{filename}_matches.pdf", dpi=300, bbox_inches="tight"
        )
    plt.show()

In [ ]:
from eval_unreal_simple import process_config

# Extractors and matchers base configurations
extractor_configs = {
    "SuperPoint": {"class": "SuperPoint"},
    "SIFT": {"class": "Sift"},
    "ORB": {"class": "Orb"},
    "AKAZE": {"class": "Akaze"},
    "BRISK": {"class": "Brisk"},
}

matcher_configs = {
    "LightGlue": {"class": "LightGlue"},
    "SuperGlue": {"class": "SuperGlue"},
    "BruteForce": {"class": "BruteForceMatcher"},
    "Flann": {"class": "FlannMatcher"},
}

# Generate compatible combinations using process_config
combinations = []
for e_name in extractor_configs.keys():
    if e_name not in extractor_configs:
        continue
    for m_name in matcher_configs.keys():
        if m_name not in matcher_configs:
            continue

        e_cfg = extractor_configs[e_name]
        m_cfg = matcher_configs[m_name]
        e_cfg_processed, m_cfg_processed = process_config(e_name, e_cfg, m_name, m_cfg)

        if e_cfg_processed is not None and m_cfg_processed is not None:
            combinations.append((e_name, e_cfg_processed, m_name, m_cfg_processed))

# Create extractors and matchers from processed configs
# Create per combination to handle cases where extractor config might differ per matcher
extractors = {}
matchers = {}

for e_name, e_cfg, m_name, m_cfg in combinations:
    combo_key = (e_name, m_name)

    # Create extractor for this specific combination (config may vary per matcher)
    if combo_key not in extractors:
        extractors[combo_key] = pnt.FeatureExtractor.from_config(e_cfg)

    # Create matcher for this specific combination (config may vary per extractor)
    if combo_key not in matchers:
        matchers[combo_key] = pnt.FeatureMatcher.from_config(m_cfg)

In [ ]:
for i, (e_name, e_cfg, m_name, m_cfg) in enumerate(combinations):
    print(f"{i}: {e_name} + {m_name}")

In [ ]:
cam_name = "front_left"
e_name = "SuperPoint"
m_name = "SuperGlue"

img_data1 = dataset[114]["cameras"][cam_name]
img_data2 = dataset[124]["cameras"][cam_name]

# Use extractor for this specific combination
feats1 = extractors[(e_name, m_name)].extract(img_data1.rgb)
feats2 = extractors[(e_name, m_name)].extract(img_data2.rgb)

feats1 = filter_features_by_depth(feats1, img_data1)
feats2 = filter_features_by_depth(feats2, img_data2)

matches = matchers[(e_name, m_name)].match(feats1, feats2)

filename = "extracted_features_consecutive"
plot_features(feats1, feats2, img_data1, img_data2, matches, filename)

In [ ]:
cam_name = "front_left"
e_name = "SuperPoint"
m_name = "SuperGlue"
filename = "extracted_features_nonconsecutive"

img_data1 = dataset[114]["cameras"][cam_name]
img_data2 = dataset[180]["cameras"][cam_name]

# Use extractor for this specific combination
combo_key = (e_name, m_name)
feats1 = extractors[combo_key].extract(img_data1.rgb)
feats2 = extractors[combo_key].extract(img_data2.rgb)

feats1 = filter_features_by_depth(feats1, img_data1)
feats2 = filter_features_by_depth(feats2, img_data2)

matches = matchers[(e_name, m_name)].match(feats1, feats2)
plot_features(feats1, feats2, img_data1, img_data2, matches, filename)

In [ ]:
cam_name = "front_left"
filename = "extracted_features_nonconsecutive"
img_data1 = dataset[114]["cameras"][cam_name]
img_data2 = dataset[124]["cameras"][cam_name]

fig, axes = plt.subplots(4, 3, figsize=(10, 7))


h, w = img_data1["rgb"].shape[:2]
ws = 10
sep = np.ones((h, ws, 3))

plt.sca(axes.flat[0])
plt.imshow(np.hstack([img_data1["rgb"], sep, img_data2["rgb"]]))
plt.axis("off")
plt.xlim(0, 2 * w + ws)
plt.ylim(h, 0)
plt.title("RGB")

np.random.seed(0)

for i, (e_name, e_cfg, m_name, m_cfg) in enumerate(combinations):
    # Feature extraction and matching
    # Use extractor for this specific combination
    feats1 = extractors[(e_name, m_name)].extract(img_data1.rgb)
    feats2 = extractors[(e_name, m_name)].extract(img_data2.rgb)
    feats1 = filter_features_by_depth(feats1, img_data1)
    feats2 = filter_features_by_depth(feats2, img_data2)
    matches = matchers[(e_name, m_name)].match(feats1, feats2)
    print(f"{i}: {e_name} + {m_name}")

    # Check valid feature coordinates in image 1
    h, w = img_data1.rgb.shape[:2]
    u1 = feats1.uv[:, 0].astype(np.int32).clip(0, w - 1)
    v1 = feats1.uv[:, 1].astype(np.int32).clip(0, h - 1)
    depth1 = img_data1.depth[v1, u1]

    # Convert to 3D world coordinates
    xyz_c1 = pnt.uv_to_xyz(feats1.uv, depth1, img_data1.intrinsics)
    xyz1_w = pnt.apply_transform(img_data1.world_T_cam, xyz_c1)

    # Project to image 2
    uv2_true, depth2 = pnt.xyz_to_uv(
        xyz1_w, img_data2.intrinsics, img_data2.world_T_cam, return_depth=True
    )

    # Compute pairwise distances between uv2_true and feats2.uv
    all_dists = np.linalg.norm(uv2_true[:, None, :] - feats2.uv[None, :, :], axis=2)
    dist_feats2true = np.min(all_dists, axis=0)

    # Match distance
    gt_thresh = 0.01 * w
    dist_match = dist_feats2true[matches.indexes[:, 1]]
    dists_normed = np.clip(dist_match / gt_thresh / 2.0, 0, 1)
    colors = np.vstack([dists_normed, 1 - dists_normed, np.zeros(len(dists_normed))]).T

    # Subsample matches
    # Sort deterministically: by distance first, then by match index to break ties
    n_matches = len(matches.indexes)
    # Use lexsort for deterministic tie-breaking: sort by distance, then by index
    sort_keys = (matches.distances, np.arange(len(matches.distances)))
    idxs_match = np.lexsort(sort_keys)[:n_matches]
    idxs_uv1, idxs_uv2 = matches.indexes[idxs_match].T

    plt.sca(axes.flat[i + 1])
    plt.imshow(np.hstack([img_data1["rgb"], sep, img_data2["rgb"]]))
    plt.axis("off")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.title(f"{e_name} + {m_name}")

    for k, (i, j) in enumerate(zip(idxs_uv1, idxs_uv2)):
        pt1, pt2 = feats1.uv[i], feats2.uv[j]
        plt.plot(
            [pt1[0], pt2[0] + w + ws],
            [pt1[1], pt2[1]],
            "-",
            color=colors[idxs_match[k]],
            lw=1.0,
            zorder=15,
        )
    plt.axis("off")
    plt.plot([], [], color="lime", lw=2.0, label="Low Error")
    plt.plot([], [], color="red", lw=2.0, label="High Error")
    plt.xlim(0, 2 * w + ws)
    plt.ylim(h, 0)
    plt.title(f"{e_name} + {m_name}")
    # plt.legend(loc="upper right")
plt.tight_layout()
plt.savefig(
    output_dir / f"figures/feature_matching_all.pdf", dpi=300, bbox_inches="tight"
)
plt.show()

In [ ]:
# Plot just the feature extractors (all detectors on their own keypoints and scores, no matching)
fig, axes = plt.subplots(2, 3, figsize=(8, 5))
axes = axes.flatten()

h, w = img_data1.rgb.shape[:2]

plt.sca(axes[0])
plt.imshow(img_data1["rgb"])
plt.xlim(0, w)
plt.ylim(h, 0)
plt.title("RGB")
plt.axis("off")

# Then each extractor
plotted_extractors = set()
for i, ((e_name, m_name), extractor) in enumerate(extractors.items()):
    if e_name in plotted_extractors:
        continue
    print(f"{i}: {e_name} + {m_name}")
    feats = extractor.extract(img_data1.rgb)
    plt.sca(axes[len(plotted_extractors) + 1])
    plt.imshow(img_data1["rgb"])
    plt.scatter(feats.uv[:, 0], feats.uv[:, 1], color="cyan", s=2)
    plt.xlim(0, w)
    plt.ylim(h, 0)
    plt.title(f"{e_name}")
    plt.axis("off")
    plotted_extractors.add(e_name)

plt.tight_layout()
plt.savefig(
    output_dir / f"figures/features_all_extractors.pdf", dpi=200, bbox_inches="tight"
)
plt.show()

In [ ]:
lusnar_dataset_config = {
    "inherit_from": "lusnar.yaml",
    "preload": "none",
}
lusnar_dataset = pnt.Dataset.from_config(lusnar_dataset_config)

In [ ]:
lusnar_dataset[0]["cameras"]

In [ ]:
cam_name = "front_left"
e_name = "ORB"
m_name = "Flann"
filename = "tmp"

img_data1 = lusnar_dataset[0]["cameras"][cam_name]
img_data2 = lusnar_dataset[1]["cameras"][cam_name]

# Use extractor for this specific combination
combo_key = (e_name, m_name)
feats1 = extractors[combo_key].extract(img_data1.rgb)
feats2 = extractors[combo_key].extract(img_data2.rgb)

feats1 = filter_features_by_depth(feats1, img_data1)
feats2 = filter_features_by_depth(feats2, img_data2)

matches = matchers[(e_name, m_name)].match(feats1, feats2)
plot_features(feats1, feats2, img_data1, img_data2, matches, filename)